In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window

In [0]:
path_raw = "dbfs:/FileStore/clientes_sinteticos.csv"

In [0]:
df_clientes = spark.read.csv(path_raw, header=True, sep=",")

## Pergunta 1: Top 5 clientes

In [0]:
"""
Resposta - O top 5 dos clientes são:
+-----------+---------------+----------------+
|cod_cliente|nm_cliente     |qtd_atualizacoes|
+-----------+---------------+----------------+
|396        |Donald Walker  |5               |
|396        |Shelia Lara    |5               |
|396        |Jennifer Malone|5               |
|396        |Sarah Allen    |5               |
|396        |Shelby Nelson  |5               |
+-----------+---------------+----------------+
"""

window_spec = Window.partitionBy("cod_cliente")

df_top5_clientes = (
    df_clientes.withColumn("qtd_atualizacoes", count("dt_atualizacao").over(window_spec).distinct())
      .orderBy(desc("qtd_atualizacoes"))
      .select("cod_cliente", "nm_cliente", "qtd_atualizacoes")
      .limit(5)
)

df_top5_clientes.show(truncate=False)

+-----------+---------------+----------------+
|cod_cliente|nm_cliente     |qtd_atualizacoes|
+-----------+---------------+----------------+
|396        |Donald Walker  |5               |
|396        |Shelia Lara    |5               |
|396        |Jennifer Malone|5               |
|396        |Sarah Allen    |5               |
|396        |Shelby Nelson  |5               |
+-----------+---------------+----------------+



## Pergunta 2: Media de idade dos clientes

In [0]:
#Verificar repetição do cod_cliente
df_clientes.filter(col("cod_cliente") == 396).display()

cod_cliente,nm_cliente,nm_pais_cliente,nm_cidade_cliente,nm_rua_cliente,num_casa_cliente,telefone_cliente,dt_nascimento_cliente,dt_atualizacao,tp_pessoa,vl_renda
396,Donald Walker,Cayman Islands,New Daisy,Palmer Crescent,25702,723.957.2526x786,1946-03-24,2024-07-18,PJ,20094.37
396,Jennifer Malone,Korea,Port Victormouth,Smith Walk,91710,(25)32909-5910,2005-11-20,2024-06-23,PJ,93709.69
396,Sarah Allen,Seychelles,North Melissabury,Burke Grove,249,(39)32960-8465,2001-10-12,2025-04-02,PJ,37091.67
396,Shelby Nelson,Romania,North Kendra,Thompson Isle,7982,(41)74353-8054,1999-03-10,2024-08-07,PF,5170.06
396,Shelia Lara,France,Lake Kennethtown,Michele Pines,133,(11)82513-7050,1990-07-11,2024-04-10,PJ,55683.41


In [0]:
#Resposta - A média de idade dos clientes é 50 anos.

media_idade = (
    df_clientes
        .withColumn("idade", datediff(current_date(), col("dt_nascimento_cliente")) / 365.25)
        .agg(round(avg("idade"),0).alias("media_idade"))
        .collect()[0]["media_idade"]
)

print(media_idade)

50.0
